In [ ]:
!pip install transformers
!pip install emoji
import torch
import transformers
import emoji
import os
import re
import copy
import numpy as np
import pandas as 
from sklearn.metrics import f1_score
from torch.utils import data
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import xgboost as xgb
import tensorflow as tf
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer

     |████████████████████████████████| 2.5MB 6.8MB/s 
     |████████████████████████████████| 3.3MB 18.8MB/s 
     |████████████████████████████████| 901kB 44.2MB/s 
     |████████████████████████████████| 133kB 7.8MB/s 


In [ ]:
#check versi pytorch
#print(torch.__version__)

1.9.0+cu102


In [ ]:
#check versi python
#!python --version

Python 3.7.10


In [ ]:
#set seed value
os.environ['PYTHONHASHSEED'] = str(1141)
np.random.seed = 1141
torch.manual_seed= 1141
tf.random.set_seed(1141)

In [ ]:
#load tokenizer for DistilBERT
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("/content/drive/MyDrive/wordpiecev2")

In [ ]:
#load pretrained DistilBERT
model_class = transformers.DistilBertModel
model = model_class.from_pretrained('/content/drive/MyDrive/model_outputv10',) #hasil training distilbert

Some weights of the model checkpoint at /content/drive/MyDrive/model_outputv10 were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#list stopwords

stopword =["yang","kan","yg","di","aja","ada","agak","sih","dari","kalo","atau","nya","udh","lah","new","itu","saja","nih","mah","kok","saja","lagi","lg","gini",
            "kl","tau","atau","pas","amp","kok","tuh","dah","jg","gini","klo","jika","pas","mana","si","utk","lain","dgn","kali","gitu","deh","pun", "adalah",
            "memang","lalu","kek","jd","sm"] 



2


In [ ]:
#load classification data
df = pd.read_csv("/content/drive/MyDrive/labaled0_data.csv")
df_text_label = df[['tweet','sentiment'] ]
df_text_label['sentiment'].value_counts()

#1 : Negative, 2 : Neutral, 3: Positive

1    2058
2     664
3     380
Name: sentiment, dtype: int64

In [ ]:
#seperate positive, neutral, and negative data
df_positif= df_text_label.loc[df_text_label['sentiment'] == 3]
df_negatif= df_text_label.loc[df_text_label['sentiment'] == 1]
df_netral= df_text_label.loc[df_text_label['sentiment'] == 2]
df_positif.shape

(380, 2)

In [ ]:
#balanced the data for each label
df_negatif_380 = df_negatif[:380]
df_netral_380 = df_netral[:380]
df_p_n = pd.concat([df_positif, df_negatif_380], ignore_index=True)
df_p_n.shape

(760, 2)

In [ ]:
#''.join( (emoji.get_emoji_regexp().sub(u'', character) for character in item.split()))

In [ ]:
#remove stopwords and Tokenization
regex = re.compile('[^a-zA-Z0-9]')
#change df_p_n to df_text_label
df_p_n['tweet']= df_p_n['tweet'].str.lower().str.split()
df_p_n['tweet'] = df_p_n['tweet'].apply(lambda x: [ regex.sub('', item)   for item in x if item not in stopword])

df_p_n['tweet'] = df_p_n['tweet'].apply( lambda x: ' '.join(x))
tokenized = df_p_n['tweet'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
print(tokenized[0])

[2, 26337, 50190, 22471, 1886, 9063, 626, 5372, 1192, 717, 1428, 51245, 37557, 1886, 677, 3]


In [ ]:
#padding
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
# masking, create an array of 0s (pad token) and 1s (real token)
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(760, 57)

In [ ]:
input_ids = torch.tensor(padded)  
input_ids = input_ids.to(torch.long) #konversi tipe IntTensor -> Long
attention_mask =  torch.from_numpy(attention_mask) 
 



In [ ]:

#take tokenized sentences as input and the last hidden state will be saved as feature
with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
#save feature
features = last_hidden_states[0][:,0,:].numpy()
features_shape3= last_hidden_states[0].numpy()

In [ ]:
#Encode with One-Hot-Encoding

from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
#save feature
features = last_hidden_states[0][:,0,:].numpy()
features_shape3= last_hidden_states[0].numpy()

#lables - Target 
labels = df_p_n['sentiment']
NNlabels= df_p_n['sentiment'].map(str)
label_encoder = LabelEncoder()
#transform label
label_encoder.fit(NNlabels)
NNlabels = label_encoder.transform(NNlabels)
NNlabels = np_utils.to_categorical(NNlabels)


TRAINING & EVALUATING CLASSIFIER

In [ ]:
#split data training

train_features, test_features, train_labels, test_labels = train_test_split(features, labels,random_state=42,stratify=labels)
NNtrain_features, NNtest_features, NNtrain_labels, NNtest_labels = train_test_split(features, NNlabels,random_state=42,stratify=NNlabels)
Bitrain_features, Bitest_features, Bitrain_labels, Bitest_labels = train_test_split(features_shape3, NNlabels,random_state=42,stratify=NNlabels)
accuracy_list=[]

# **LOGISTIC REGRESSION**

In [ ]:
#LOGISTIC REGRESSION
lr_clf = LogisticRegression(multi_class = 'auto', C=7.2, solver='lbfgs',max_iter=100,)
lr_clf.fit(train_features, train_labels)


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=7.2, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
#GRID SEARCH LOGISTIC REGRESSION
parameters = {'solver' : ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 'C':[1,3,7,10], 'multi_class' : ['auto', 'ovr', 'multinomial']}

lr = LogisticRegression()
clf = GridSearchCV(lr, parameters)
clf.fit(train_features, train_labels)
sorted(clf.cv_results_.keys())

/usr/local/lib/python3.7/dist-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/optimize.py:204: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/usr/local/lib/python3.7/dist-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/optimize.py:204: UserWarning: Line Search failed
  warnings.warn('Line Search failed')
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/prepr

KeyboardInterrupt: ignored

In [ ]:
#print(clf.cv_results_) #the result saved in clf.cv_results_

In [ ]:
print(clf.cv_results_['params'][7]) 

In [ ]:

a=[]
df_grid_logistic= pd.DataFrame()    
for i in range(0,60):
  a.append(list(clf.cv_results_['params'][i].values()))
  a[-1].append(clf.cv_results_['mean_test_score'][i])

In [ ]:
#Save in csv
df_grid_logistic= pd.DataFrame(data=a)
df_grid_logistic.to_csv("logistic_120mb_stopword_3target.csv",index=False)


In [ ]:
# test model
accuracy_list.append(lr_clf.score(test_features, test_labels))
lr_clf.score(test_features, test_labels)


# **XGBOOST**

In [ ]:
#XGBoost
clf = XGBClassifier()
clf.fit(train_features, train_labels)

#test
y_pred = clf.predict(test_features)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(test_labels, predictions)
accuracy_list.append(float("{:.2f}".format(accuracy * 100.0)))
print("Accuracy: %.2f%%" % (accuracy * 100.0))



In [ ]:
#XGBoost grid search
XGBparameters = {
        
              'learning_rate': [0.2,0.1, 0.05],
              'max_depth': [2,3,4,5],
              'n_estimators': [10,50,100], 
              }

clf_grid = XGBClassifier( )
grid_xgb = GridSearchCV(estimator=clf_grid, param_grid=XGBparameters)
grid_xgb.fit(train_features, train_labels)

In [ ]:
#Save in CSV
temp_xgb_grid=[]
for i in range(0,36):
  temp_xgb_grid.append(list(grid_xgb.cv_results_['params'][i].values()))
  temp_xgb_grid[-1].append(grid_xgb.cv_results_['mean_test_score'][i])

df_grid_xgb= pd.DataFrame(data=temp_xgb_grid)
df_grid_xgb.to_csv("xgb_120mb_stopword_3target.csv",index=False)

# **SVM**

In [ ]:
#SVM grid search
from sklearn.svm import SVC
parameters = {'kernel':['linear', 'rbf','poly'], 'C':[1,5, 10],'gamma': [ 0.1,0.05,0.02]}
svc = SVC()
clf_svm = GridSearchCV(svc, parameters)
clf_svm.fit(train_features, train_labels)


In [ ]:
#Save in CSV
temp_svm_grid=[]
for i in range(0,27):
  temp_svm_grid.append(list(clf_svm.cv_results_['params'][i].values()))
  temp_svm_grid[-1].append(clf_svm.cv_results_['mean_test_score'][i])
df_grid_svm= pd.DataFrame(data=temp_svm_grid)
df_grid_svm.to_csv("cek_svm_120mb_nostopword_3target.csv",index=False)

In [ ]:
#SVM manual
from sklearn.svm import SVC
svm_model_linear = SVC(kernel = 'rbf', C = 11, gamma= 0.023,).fit(train_features, train_labels)
accuracy_svm = svm_model_linear.score(test_features, test_labels)
accuracy_list.append(float("{:.2f}".format(accuracy_svm * 100.0)))
print(accuracy_svm) 

In [ ]:
b =  svm_model_linear.predict(test_features)
print(b)
df_predict = pd.DataFrame({'col':b})
df_text_label["predict"] = df_predict
df_text_label.to_csv("svm_prediction3.csv",index=False)

In [ ]:
df_text_label.to_csv("svm_prediction3.csv",index=False)

# **DNN (Simple Neural Network)**

In [ ]:
#DNN
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
model = Sequential([
    Dense(768, input_shape=(train_features.shape[1],), activation='relu'),
   
    Dense(180, activation='relu'),
    
    Dense(NNtrain_labels.shape[1], activation='softmax'),
])

In [ ]:
print(model.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 768)               590592    
_________________________________________________________________
dense_8 (Dense)              (None, 180)               138420    
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 362       
Total params: 729,374
Trainable params: 729,374
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#Train DNN
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(NNtrain_features, NNtrain_labels, epochs=600, batch_size=64, validation_split=0.2)

In [ ]:
#test predict
to_label=[1,2,3] #<- adjust it with the testing
pred = model.predict(test_features) 

#print(classification_report(test_labels, pred))
pred_label=[]
for i in pred:
  pred_label.append(to_label[np.argmax(i)])
  
df_pred = pd.DataFrame(data=pred_label)

NNlabel = []
for i in NNtest_labels:
  NNlabel.append(to_label[np.argmax(i)])
  
df_labelNN = pd.DataFrame(data=NNlabel)

print(f1_score(test_labels, df_pred[0]
, average='micro'))

In [ ]:
#simpan model
#model.save('/content/drive/MyDrive/nn_acc_42_45')

In [ ]:
df_pred.value_counts()

3    115
1     75
dtype: int64

# **BI-LSTM**

In [ ]:
Bitrain_features.shape

In [ ]:
#BiLISTM
from keras.layers.recurrent import LSTM, GRU
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import  Bidirectional, SpatialDropout1D,TimeDistributed,Flatten

train_features3, test_features3, train_labels3, test_labels3 = train_test_split(features_shape3, labels)
modelBiLSTM = Sequential([
   
    Bidirectional(LSTM(16, dropout=0, recurrent_dropout=0,return_sequences=True)),
    Bidirectional(LSTM(4, dropout=0, recurrent_dropout=0,return_sequences=False)),
  
    Flatten(),

    Dense(NNtrain_labels.shape[1], activation='softmax'),
])



modelBiLSTM.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history2 = modelBiLSTM.fit(Bitrain_features, Bitrain_labels, epochs=1, batch_size=64,validation_split=0.2)

In [ ]:
Bilabel = []
for i in Bitest_labels:
  Bilabel.append(to_label[np.argmax(i)])
  
df_labelBi = pd.DataFrame(data=Bilabel)

In [ ]:
#predict
to_label=[1,3] #<- adjust with the current testing
predBi = modelBiLSTM.predict(Bitest_features) 


pred_label=[]
for i in predBi:
  pred_label.append(to_label[np.argmax(i)-1])
  
df_predBi = pd.DataFrame(data=pred_label)


from sklearn.metrics import f1_score
print(f1_score(test_labels, df_predBi[0]
, average='micro')) 
